In [ ]:
from urllib import request
from bs4 import BeautifulSoup
import json

num = 0
title_list = []; writer_list = []; genre_list = []; category_list = []; introduction_list = []

def getLists(url):
    global num
    html = request.urlopen(url).read()
    soup = BeautifulSoup(html,'html.parser')
    jsondata = json.loads(soup.text)
    
    for i in jsondata['data']:
        t = i.get('title')
        if(t in title_list):
            continue
        else:
            title_list.append(t)
    
            writers = []
            for j in i.get('cartoon').get('artists'):
                writers.append(j.get('name'))
            writer_list.append(writers)
    
            genres = []
            for j in i.get('cartoon').get('genres'):
                genres.append(j.get('name'))
            genre_list.append(genres)
    
            categories = []
            for j in i.get('cartoon').get('categories'):
                categories.append(j.get('name'))
            category_list.append(categories)
    
            introduction_list.append(i.get('introduction'))
    
            num = num + 1

url_mon = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/mon'
url_tue = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/tue'
url_wed = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/wed'
url_thu = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/thu'
url_fri = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/fri'
url_sat = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/sat'
url_sun = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/sun'
url_finished = 'http://webtoon.daum.net/data/pc/webtoon/list_daily_ranking/finished'
url_recommend = 'http://webtoon.daum.net/data/pc/webtoon/list_recommend'
url_finished_gen = 'http://webtoon.daum.net/data/pc/webtoon/list_finished/?genre_id='

url_list = [url_mon, url_tue, url_wed, url_thu, url_fri, url_sat, url_sun, url_finished, url_recommend]

for i in url_list:
    getLists(i)

In [ ]:
nickname_list = []
url='http://webtoon.daum.net/data/pc/webtoon/list_finished/?genre_id='

html = request.urlopen(url).read()
soup = BeautifulSoup(html,'html.parser')
jsondata = json.loads(soup.text)
    
for i in jsondata['data']:
    n = i.get('nickname')
    if(n in nickname_list):
        continue
    else:
        nickname_list.append(n)

for nickname in nickname_list:
    URL = 'http://webtoon.daum.net/data/pc/webtoon/view/' + nickname
    
    html = request.urlopen(URL).read()
    soup = BeautifulSoup(html,'html.parser')
    jsondata = json.loads(soup.text)
    
    status = jsondata['result'].get('status')
    if(status == '403'):
        continue
    else:
        
        i = jsondata['data'].get('webtoon')
    
        t = i.get('title')
        if(t in title_list):
            continue
        else:
            title_list.append(t)
            
            writers = []
            for j in i.get('cartoon').get('artists'):
                writers.append(j.get('name'))
            writer_list.append(writers)
    
            genres = []
            for j in i.get('cartoon').get('genres'):
                genres.append(j.get('name'))
            genre_list.append(genres)
    
            categories = []
            for j in i.get('cartoon').get('categories'):
                categories.append(j.get('name'))
            category_list.append(categories)
    
            introduction_list.append(i.get('introduction'))
    
            num = num + 1

In [ ]:
import pandas as pd

import pandas as pd

webtoonfile = pd.DataFrame({'제목': title_list, \
                            '작가': writer_list, \
                            '장르': genre_list, \
                            '카테고리': category_list, \
                            '한줄 스토리': introduction_list})

webtoonfile.to_csv('daum_webtoon_crawling.csv')

In [ ]:
webtoonfile.head()